<a href="https://colab.research.google.com/github/rmbayer/ASL_NN/blob/master/asl_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CNN TensorFlow Framework for American Sign Language Detection
### **Bob Bayer and Jonathan Rice 2020**

**Retrieve Datasets**

In [6]:
# Import and mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [8]:
# Check drive contents
!ls "/content/drive/My Drive/Sign Language"

 asl-alphabet.zip		      'Sign Language Project Research.gdoc'
'ASL Recognition with Deep Learning'


**The following python code will use the OS library to use Operating System libraries, giving you access to the file system, and the zipfile library allowing you to unzip the data.**

In [0]:
import os
import zipfile

local_zip = '/content/drive/My Drive/Sign Language/asl-alphabet.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
# Extract to a local file and close the connection to the drive file
zip_ref.extractall('/tmp/') 
zip_ref.close()

**Set the directories for the training and test datasets**

In [0]:
train_dir = os.path.join('/tmp/asl_alphabet_train/asl_alphabet_train')
test_dir= os.path.join('/tmp/asl_alphabet_test/asl_alphabet_test')

**Veryify the subfolders of the directory**

In [65]:
train_names = os.listdir(train_dir)
print(train_names)
test_names = os.listdir(test_dir)
print(test_names)

['A', 'G', 'S', 'W', 'N', 'L', 'H', 'X', 'K', 'V', 'P', 'I', 'T', 'B', 'C', 'R', 'del', 'J', 'F', 'D', 'Z', 'space', 'E', 'Q', 'M', 'U', 'Y', 'nothing', 'O']
['J_test.jpg', 'N_test.jpg', 'P_test.jpg', 'X_test.jpg', 'A_test.jpg', 'T_test.jpg', 'Z_test.jpg', 'G_test.jpg', 'H_test.jpg', 'C_test.jpg', 'Y_test.jpg', 'W_test.jpg', 'E_test.jpg', 'space_test.jpg', 'U_test.jpg', 'K_test.jpg', 'D_test.jpg', 'M_test.jpg', 'S_test.jpg', 'B_test.jpg', 'R_test.jpg', 'L_test.jpg', 'V_test.jpg', 'I_test.jpg', 'nothing_test.jpg', 'F_test.jpg', 'O_test.jpg', 'Q_test.jpg']


str

**Build the Model**

In [0]:
import tensorflow as tf

In [0]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 128 neuron hidden layer
    tf.keras.layers.Dense(128, activation = tf.nn.relu),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(29, activation = tf.nn.softmax)
])

**Inspect the Layers**

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 64)        3

**Set the optimizer** <br>
Use adam for it's adaptive learning rate

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

**Define the ImageGenerator**

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(200, 200),
        batch_size=100,
        # Since we use categorical_crossentropy loss, we need sparse labels
        class_mode='sparse')

# May not need this
label_map = (train_generator.class_indices)



Found 87000 images belonging to 29 classes.


In [33]:
print(label_map)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28}


**Train**

In [17]:
model.fit(train_generator, 
          steps_per_epoch = 870, #we have 87,000 images, batch size of 100
          epochs=10)

Epoch 1/10
870/870 [==============================] - 142s 163ms/step - loss: 0.9226 - accuracy: 0.7188
Epoch 2/10
870/870 [==============================] - 141s 163ms/step - loss: 0.1094 - accuracy: 0.9636
Epoch 3/10
870/870 [==============================] - 139s 160ms/step - loss: 0.0547 - accuracy: 0.9828
Epoch 4/10
870/870 [==============================] - 139s 160ms/step - loss: 0.0377 - accuracy: 0.9883
Epoch 5/10
870/870 [==============================] - 139s 160ms/step - loss: 0.0304 - accuracy: 0.9906
Epoch 6/10
870/870 [==============================] - 138s 159ms/step - loss: 0.0277 - accuracy: 0.9915
Epoch 7/10
870/870 [==============================] - 138s 159ms/step - loss: 0.0178 - accuracy: 0.9947
Epoch 8/10
870/870 [==============================] - 137s 158ms/step - loss: 0.0189 - accuracy: 0.9945
Epoch 9/10
870/870 [==============================] - 139s 159ms/step - loss: 0.0181 - accuracy: 0.9941
Epoch 10/10
870/870 [==============================] - 139s 159m

In [74]:
import numpy as np
from keras.preprocessing import image

for pic in test_names:

  # predicting images
  path = test_dir + '/' + pic
  img = image.load_img(path, target_size=(200, 200, 3))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0) # I'm not sure why you need this part

  y_prob = model.predict(x)
  y_classes = y_prob.argmax(axis=-1)
  #print(y_classes)
  print('Actual: ' + str(pic) + '   Predicted: ' + sorted(train_names)[int(y_classes)])

  #classes = np.argmax(model.predict(images), axis=-1)
  #print(classes[0])


Actual: J_test.jpg   Predicted: J
Actual: N_test.jpg   Predicted: N
Actual: P_test.jpg   Predicted: P
Actual: X_test.jpg   Predicted: L
Actual: A_test.jpg   Predicted: A
Actual: T_test.jpg   Predicted: T
Actual: Z_test.jpg   Predicted: Z
Actual: G_test.jpg   Predicted: G
Actual: H_test.jpg   Predicted: H
Actual: C_test.jpg   Predicted: C
Actual: Y_test.jpg   Predicted: Y
Actual: W_test.jpg   Predicted: W
Actual: E_test.jpg   Predicted: E
Actual: space_test.jpg   Predicted: space
Actual: U_test.jpg   Predicted: U
Actual: K_test.jpg   Predicted: K
Actual: D_test.jpg   Predicted: D
Actual: M_test.jpg   Predicted: M
Actual: S_test.jpg   Predicted: L
Actual: B_test.jpg   Predicted: B
Actual: R_test.jpg   Predicted: R
Actual: L_test.jpg   Predicted: L
Actual: V_test.jpg   Predicted: V
Actual: I_test.jpg   Predicted: I
Actual: nothing_test.jpg   Predicted: nothing
Actual: F_test.jpg   Predicted: F
Actual: O_test.jpg   Predicted: O
Actual: Q_test.jpg   Predicted: Q


***Debugging***

In [25]:
#!ls '/tmp/'
#!ls '/tmp/asl-alphabet/'
#!ls '/tmp/asl_alphabet_train/'
#!ls '/tmp/asl_alphabet_train/asl_alphabet_train'
!ls '/tmp/asl_alphabet_test/asl_alphabet_test'

A_test.jpg  G_test.jpg	M_test.jpg	  R_test.jpg	  W_test.jpg
B_test.jpg  H_test.jpg	nothing_test.jpg  space_test.jpg  X_test.jpg
C_test.jpg  I_test.jpg	N_test.jpg	  S_test.jpg	  Y_test.jpg
D_test.jpg  J_test.jpg	O_test.jpg	  T_test.jpg	  Z_test.jpg
E_test.jpg  K_test.jpg	P_test.jpg	  U_test.jpg
F_test.jpg  L_test.jpg	Q_test.jpg	  V_test.jpg
